# [ 5-2. 매출 모델링 ]

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import cafle as cf
from cafle import Index, Account
from cafle import Setattr

In [2]:
from practice.astn0_overview import overview, idx

## 1. 분양매출 테이블 작성

In [3]:
분양_오피스텔 = DataFrame({
    "호실면적" : [10, 11, 12, 13], #평
    "호실수"  : [120, 30, 120, 50], #실
    "평단가"  : [20.0, 20.0, 20.0, 20.0], #백만원/평
},  index   = ['A', 'B', 'C', 'D'])

In [4]:
분양_오피스텔

,호실면적,호실수,평단가
A,10,120,20.0
B,11,30,20.0
C,12,120,20.0
D,13,50,20.0


In [6]:
B = 분양_오피스텔
B['호실면적m2'] = round(B['호실면적'] * cf.PY, 2)
B['면적소계'] = B['호실면적'] * B['호실수']
B['면적소계m2'] = round(B['호실면적m2'] * B['호실수'], 2)
B['호실단가'] = B['호실면적'] * B['평단가']
B['매출소계'] = B['면적소계'] * B['평단가']

In [7]:
분양_오피스텔

,호실면적,호실수,평단가,호실면적m2,면적소계,면적소계m2,호실단가,매출소계
A,10,120,20.0,3.03,1200,363.6,200.0,24000.0
B,11,30,20.0,3.33,330,99.9,220.0,6600.0
C,12,120,20.0,3.63,1440,435.6,240.0,28800.0
D,13,50,20.0,3.93,650,196.5,260.0,13000.0


In [8]:
분양_오피스텔.sum()

호실면적         46.00
호실수         320.00
평단가          80.00
호실면적m2       13.92
면적소계       3620.00
면적소계m2     1095.60
호실단가        920.00
매출소계      72400.00
dtype: float64

In [9]:
print(f"오피스텔 매출액 : {sum(분양_오피스텔['매출소계']):,.0f}백만원")

오피스텔 매출액 : 72,400백만원


In [10]:
분양_근생 = DataFrame({
    "면적" : [140], #평
    "평단가" : [40.0], #백만원/평
},  index = ['F1'])

In [12]:
B = 분양_근생
B['면적m2'] = round(B['면적'] * cf.PY, 2)
B['매출소계'] = B['면적'] * B['평단가']

In [13]:
분양_근생

,면적,평단가,면적m2,매출소계
F1,140,40.0,42.35,5600.0


In [14]:
print(f"오피스텔 매출액 : {sum(분양_오피스텔['매출소계']):,.0f}백만원")
print(f"근생 매출액    : {sum(분양_근생['매출소계']):,.0f}백만원")
print(f"총 매출액     : {sum(분양_오피스텔['매출소계']) + sum(분양_근생['매출소계']):,.0f}백만원")

오피스텔 매출액 : 72,400백만원
근생 매출액    : 5,600백만원
총 매출액     : 78,000백만원


In [15]:
분양테이블 = {
    "오피" : 분양_오피스텔,
    "근생" : 분양_근생,
}

In [16]:
분양테이블

{'오피':    호실면적  호실수   평단가  호실면적m2  면적소계  면적소계m2   호실단가     매출소계
 A    10  120  20.0    3.03  1200   363.6  200.0  24000.0
 B    11   30  20.0    3.33   330    99.9  220.0   6600.0
 C    12  120  20.0    3.63  1440   435.6  240.0  28800.0
 D    13   50  20.0    3.93   650   196.5  260.0  13000.0,
 '근생':      면적   평단가   면적m2    매출소계
 F1  140  40.0  42.35  5600.0}

In [17]:
분양테이블['오피']

,호실면적,호실수,평단가,호실면적m2,면적소계,면적소계m2,호실단가,매출소계
A,10,120,20.0,3.03,1200,363.6,200.0,24000.0
B,11,30,20.0,3.33,330,99.9,220.0,6600.0
C,12,120,20.0,3.63,1440,435.6,240.0,28800.0
D,13,50,20.0,3.93,650,196.5,260.0,13000.0


In [18]:
분양테이블['근생']

,면적,평단가,면적m2,매출소계
F1,140,40.0,42.35,5600.0


In [19]:
분양매출 = {
    "오피" : 분양테이블['오피']['매출소계'].sum(),
    "근생" : 분양테이블['근생']['매출소계'].sum(),
    "합계" : 분양테이블['오피']['매출소계'].sum() + 분양테이블['근생']['매출소계'].sum()
}

In [20]:
분양매출

{'오피': 72400.0, '근생': 5600.0, '합계': 78000.0}

## 2. 분양대금 납입 일정

In [21]:
대금납입일정 = DataFrame({
    '구분': ['계약금', '중도금1', '중도금2', '중도금3', '중도금4', '잔금'],
    '오피': [   0.1,     0.1,     0.1,      0.1,     0.1,   0.5],
    '근생': [   0.1,     0.1,     0.1,      0.1,     0.0,   0.6],
}, index=  [idx.sales[0], idx.sales[5], idx.sales[10], idx.sales[15], idx.sales[20], idx.sales[-1]])

In [22]:
대금납입일정

,구분,오피,근생
2023-04-30,계약금,0.1,0.1
2023-09-30,중도금1,0.1,0.1
2024-02-29,중도금2,0.1,0.1
2024-07-31,중도금3,0.1,0.1
2024-12-31,중도금4,0.1,0.0
2025-04-30,잔금,0.5,0.6


In [23]:
대금납입일정['납입오피'] = 대금납입일정['오피'] * 분양테이블['오피']['매출소계'].sum()
대금납입일정['납입근생'] = 대금납입일정['근생'] * 분양테이블['근생']['매출소계'].sum()
대금납입일정['납입소계'] = 대금납입일정['납입오피'] + 대금납입일정['납입근생']

In [24]:
대금납입일정

,구분,오피,근생,납입오피,납입근생,납입소계
2023-04-30,계약금,0.1,0.1,7240.0,560.0,7800.0
2023-09-30,중도금1,0.1,0.1,7240.0,560.0,7800.0
2024-02-29,중도금2,0.1,0.1,7240.0,560.0,7800.0
2024-07-31,중도금3,0.1,0.1,7240.0,560.0,7800.0
2024-12-31,중도금4,0.1,0.0,7240.0,0.0,7240.0
2025-04-30,잔금,0.5,0.6,36200.0,3360.0,39560.0


## 3. 분양률 가정

In [25]:
분양률가정 = DataFrame({
    '오피': [   0.2,     0.2,     0.2,      0.2,     0.2],
    '근생': [   0.0,     0.0,     0.0,      0.0,     1.0],
}, index=  [idx.sales[0], idx.sales[6], idx.sales[12], idx.sales[18], idx.sales[-1]])

In [26]:
분양률가정['계약오피'] = 분양률가정['오피'] * 분양테이블['오피']['매출소계'].sum()
분양률가정['계약근생'] = 분양률가정['근생'] * 분양테이블['근생']['매출소계'].sum()
분양률가정['계약소계'] = 분양률가정['계약오피'] + 분양률가정['계약근생']

In [27]:
분양률가정

,오피,근생,계약오피,계약근생,계약소계
2023-04-30,0.2,0.0,14480.0,0.0,14480.0
2023-10-31,0.2,0.0,14480.0,0.0,14480.0
2024-04-30,0.2,0.0,14480.0,0.0,14480.0
2024-10-31,0.2,0.0,14480.0,0.0,14480.0
2025-04-30,0.2,1.0,14480.0,5600.0,20080.0


In [28]:
분양률가정.cumsum()

,오피,근생,계약오피,계약근생,계약소계
2023-04-30,0.2,0.0,14480.0,0.0,14480.0
2023-10-31,0.4,0.0,28960.0,0.0,28960.0
2024-04-30,0.6,0.0,43440.0,0.0,43440.0
2024-10-31,0.8,0.0,57920.0,0.0,57920.0
2025-04-30,1.0,1.0,72400.0,5600.0,78000.0


## 4. Sales Account 설정

### 1) 최초 100% 분양 가정

In [29]:
sales = Account(idx)
sales.오피 = sales.subacc('오피')
sales.근생 = sales.subacc('근생')

In [30]:
sales

Account(main, len 30, dct: ['오피', '근생'])

In [31]:
sales.오피

Account(오피, len 30)

In [32]:
sales.근생

Account(근생, len 30)

In [33]:
sales.오피.subscd(대금납입일정.index, 대금납입일정['납입오피'])
sales.근생.subscd(대금납입일정.index, 대금납입일정['납입근생'])

In [34]:
sales.오피.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,7240.0,7240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7240.0
2023-05-31,0.0,0.0,0.0,7240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7240.0
2023-06-30,0.0,0.0,0.0,7240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7240.0
2023-07-31,0.0,0.0,0.0,7240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7240.0
2023-08-31,0.0,0.0,0.0,7240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7240.0
2023-09-30,0.0,0.0,7240.0,14480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14480.0
2023-10-31,0.0,0.0,0.0,14480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14480.0


In [35]:
sales.근생.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,560.0,560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560.0
2023-05-31,0.0,0.0,0.0,560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560.0
2023-06-30,0.0,0.0,0.0,560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560.0
2023-07-31,0.0,0.0,0.0,560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560.0
2023-08-31,0.0,0.0,0.0,560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560.0
2023-09-30,0.0,0.0,560.0,1120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1120.0
2023-10-31,0.0,0.0,0.0,1120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1120.0


### 2) 분양률 가정 적용

In [36]:
sales = Account(idx)
sales.오피 = sales.subacc('오피')
sales.근생 = sales.subacc('근생')

In [37]:
현금스케줄_오피 = DataFrame(index = idx)
현금스케줄_오피['계약율'] = 분양률가정['오피']
현금스케줄_오피['납입율'] = 대금납입일정['오피']
현금스케줄_오피 = 현금스케줄_오피.fillna(0.0)

In [38]:
현금스케줄_오피

,계약율,납입율
2023-01-31,0.0,0.0
2023-02-28,0.0,0.0
2023-03-31,0.0,0.0
2023-04-30,0.2,0.1
2023-05-31,0.0,0.0
2023-06-30,0.0,0.0
2023-07-31,0.0,0.0
2023-08-31,0.0,0.0
2023-09-30,0.0,0.1
2023-10-31,0.2,0.0


In [39]:
현금스케줄_오피[['계약율누적', '납입율누적']] = 현금스케줄_오피.cumsum()
현금스케줄_오피['현금율누적'] = 현금스케줄_오피['계약율누적'] * 현금스케줄_오피['납입율누적']
현금스케줄_오피['현금율유입'] = 현금스케줄_오피['현금율누적'].diff()
현금스케줄_오피 = 현금스케줄_오피.fillna(0.0)

In [40]:
현금스케줄_오피

,계약율,납입율,계약율누적,납입율누적,현금율누적,현금율유입
2023-01-31,0.0,0.0,0.0,0.0,0.00,0.00
2023-02-28,0.0,0.0,0.0,0.0,0.00,0.00
2023-03-31,0.0,0.0,0.0,0.0,0.00,0.00
2023-04-30,0.2,0.1,0.2,0.1,0.02,0.02
2023-05-31,0.0,0.0,0.2,0.1,0.02,0.00
2023-06-30,0.0,0.0,0.2,0.1,0.02,0.00
2023-07-31,0.0,0.0,0.2,0.1,0.02,0.00
2023-08-31,0.0,0.0,0.2,0.1,0.02,0.00
2023-09-30,0.0,0.1,0.2,0.2,0.04,0.02
2023-10-31,0.2,0.0,0.4,0.2,0.08,0.04


In [41]:
sales.오피.subscd(현금스케줄_오피.index, 현금스케줄_오피['현금율유입'] * 분양테이블['오피']['매출소계'].sum())

In [42]:
sales.오피.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,1448.0,1448.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1448.0
2023-05-31,0.0,0.0,0.0,1448.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1448.0
2023-06-30,0.0,0.0,0.0,1448.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1448.0
2023-07-31,0.0,0.0,0.0,1448.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1448.0
2023-08-31,0.0,0.0,0.0,1448.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1448.0
2023-09-30,0.0,0.0,1448.0,2896.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2896.0
2023-10-31,0.0,0.0,2896.0,5792.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5792.0


In [43]:
현금스케줄_근생 = DataFrame(index = idx)
현금스케줄_근생['계약율'] = 분양률가정['근생']
현금스케줄_근생['납입율'] = 대금납입일정['근생']
현금스케줄_근생 = 현금스케줄_근생.fillna(0.0)

현금스케줄_근생[['계약율누적', '납입율누적']] = 현금스케줄_근생.cumsum()
현금스케줄_근생['현금율누적'] = 현금스케줄_근생['계약율누적'] * 현금스케줄_근생['납입율누적']
현금스케줄_근생['현금율유입'] = 현금스케줄_근생['현금율누적'].diff()
현금스케줄_근생 = 현금스케줄_근생.fillna(0.0)

In [44]:
sales.근생.subscd(현금스케줄_근생.index, 현금스케줄_근생['현금율유입'] * 분양테이블['근생']['매출소계'].sum())

In [45]:
sales.근생.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-08-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3) 분양 현금흐름 계산 함수 만들기

In [46]:
def 현금스케줄(분양률가정, 대금납입일정):
    rslt = DataFrame(index = idx)
    rslt['계약율'] = 분양률가정
    rslt['납입율'] = 대금납입일정
    rslt = rslt.fillna(0.0)
    
    rslt[['계약율누적', '납입율누적']] = rslt.cumsum()
    rslt['현금율누적'] = rslt['계약율누적'] * rslt['납입율누적']
    rslt['현금율유입'] = rslt['현금율누적'].diff()
    rslt = rslt.fillna(0.0)
    
    return rslt

In [47]:
현금스케줄_오피2 = 현금스케줄(분양률가정['오피'], 대금납입일정['오피'])

In [48]:
현금스케줄_오피2

,계약율,납입율,계약율누적,납입율누적,현금율누적,현금율유입
2023-01-31,0.0,0.0,0.0,0.0,0.00,0.00
2023-02-28,0.0,0.0,0.0,0.0,0.00,0.00
2023-03-31,0.0,0.0,0.0,0.0,0.00,0.00
2023-04-30,0.2,0.1,0.2,0.1,0.02,0.02
2023-05-31,0.0,0.0,0.2,0.1,0.02,0.00
2023-06-30,0.0,0.0,0.2,0.1,0.02,0.00
2023-07-31,0.0,0.0,0.2,0.1,0.02,0.00
2023-08-31,0.0,0.0,0.2,0.1,0.02,0.00
2023-09-30,0.0,0.1,0.2,0.2,0.04,0.02
2023-10-31,0.2,0.0,0.4,0.2,0.08,0.04


### 4) 분양률 시나리오 분석

In [49]:
분양률가정시나리오 = {}

분양률가정시나리오[1] = DataFrame({
    '오피': [   0.2,     0.2,     0.2,      0.2,     0.2],
    '근생': [   0.0,     0.0,     0.0,      0.0,     1.0],
}, index=  [idx.sales[0], idx.sales[6], idx.sales[12], idx.sales[18], idx.sales[-1]])

분양률가정시나리오[2] = DataFrame({
    '오피': [   1.0,     0.0,     0.0,      0.0,     0.0],
    '근생': [   1.0,     0.0,     0.0,      0.0,     0.0],
}, index=  [idx.sales[0], idx.sales[6], idx.sales[12], idx.sales[18], idx.sales[-1]])

분양률가정시나리오[3] = DataFrame({
    '오피': [   0.0,     0.0,     0.0,      0.0,     1.0],
    '근생': [   0.0,     0.0,     0.0,      0.0,     1.0],
}, index=  [idx.sales[0], idx.sales[6], idx.sales[12], idx.sales[18], idx.sales[-1]])

In [50]:
분양률가정 = 분양률가정시나리오[1]
분양률가정['계약오피'] = 분양률가정['오피'] * 분양테이블['오피']['매출소계'].sum()
분양률가정['계약근생'] = 분양률가정['근생'] * 분양테이블['근생']['매출소계'].sum()
분양률가정['계약소계'] = 분양률가정['계약오피'] + 분양률가정['계약근생']

In [51]:
분양률가정

,오피,근생,계약오피,계약근생,계약소계
2023-04-30,0.2,0.0,14480.0,0.0,14480.0
2023-10-31,0.2,0.0,14480.0,0.0,14480.0
2024-04-30,0.2,0.0,14480.0,0.0,14480.0
2024-10-31,0.2,0.0,14480.0,0.0,14480.0
2025-04-30,0.2,1.0,14480.0,5600.0,20080.0


## 5. Sales Account 함수 설정

In [52]:
@Setattr(sales.dct)
def get_salesamt(sls, acc, idxno):
    amt = sls.scd_out[idxno]
    sls.send(idxno, amt, acc, note=f"분양매출({sls.name})")
    return amt

In [53]:
oprtg = Account(idx)

In [54]:
sales.오피.get_salesamt(oprtg, idx[3])

1448.0000000000002

In [55]:
oprtg.df

,bal_strt,amt_in,amt_out,bal_end
2023-01-31,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0
2023-04-30,0.0,1448.0,0.0,1448.0
2023-05-31,1448.0,0.0,0.0,1448.0
2023-06-30,1448.0,0.0,0.0,1448.0
2023-07-31,1448.0,0.0,0.0,1448.0
2023-08-31,1448.0,0.0,0.0,1448.0
2023-09-30,1448.0,0.0,0.0,1448.0
2023-10-31,1448.0,0.0,0.0,1448.0
